# Rozwiązanie problemu produkcji okien

Problem: ile wyprodukować okien 

* aluminiowych
* drewnianych

żeby uzyskać maksymalny zysk.

## Ograniczenia: 

Fabryka F1 pracuje przez 4h tygodniowo produkując tylko okna aluminiowe; czas produkcji partii to 1h

Fabryka F2 pracuje przez 12h tygodniowo produkując tylko okna drewniane; czas produkcji partii to 2h

Fabryka F2 pracuje przez 18h tygodniowo produkując dowolne okna; czas produkcji partii okien drewnianych to 3h, aluminiowych - 2h

## Definicja problemu

In [5]:
from pulp import *

zyskOkna = LpProblem("Maksyamalizacja zysku z okien",LpMaximize)

#zysk za x1 to 3000, a za x2 to 5000
x1=LpVariable("oknaAluminiowe",0)
x2=LpVariable("oknaDrewniane",0)

#wspolczynnik do minimalizacji
zyskOkna += 3*x1 + 5*x2, "Zysk"

#ograniczenia
zyskOkna += x1*1 <= 4, "CzasF1"
zyskOkna += x2*2 <= 12, "CzasF2"
zyskOkna += 3*x1 + 2*x2 <= 18, "CzasF3"



## Rozwiązanie prolbemu

In [6]:
zyskOkna.solve()
print("Status:", LpStatus[zyskOkna.status])

Status: Optimal


In [7]:
for v in zyskOkna.variables():
    print(v.name, "=", v.varValue)

oknaAluminiowe = 2.0
oknaDrewniane = 6.0


## Wprowadzenie mozliwosci modyfikacji parametrow stalych
Dodanie suwaków umożliwia zmianę wprowadzanych wartości takich jak zysk z jednej partii okien drewnianych i aluminiowych

In [8]:
#wprowadzenie mozliwosci modyfikacji parametrow stalych
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

zysk_okien_aluminiowych_slider = FloatSlider(min=0,max = 10000, value= 3000, description="zysk_okien_aluminiowych")
zysk_okien_drewnianych_slider = FloatSlider(min=0,max = 10000, value= 5000, description="zysk_okien_drewnianych")

tempo_drewn_f1_slider = IntSlider(min=0,max = 5, value= 0, description="tempo_drewn_f1")
tempo_drewn_f2_slider = IntSlider(min=0,max = 5, value= 2, description="tempo_drewn_f1")
tempo_drewn_f3_slider = IntSlider(min=0,max = 5, value= 2, description="tempo_drewn_f1")

tempo_alumi_f1_slider = IntSlider(min=0,max = 5, value= 1, description="tempo_alumi_f1")
tempo_alumi_f2_slider = IntSlider(min=0,max = 5, value= 0, description="tempo_alumi_f1")
tempo_alumi_f3_slider = IntSlider(min=0,max = 5, value= 3, description="tempo_alumi_f1")

czas_f1_slider = IntSlider(min=0,max = 20, value=  4, description="czas_f1")
czas_f2_slider = IntSlider(min=0,max = 20, value= 12, description="czas_f2")
czas_f3_slider = IntSlider(min=0,max = 20, value= 18, description="czas_f3")

def parametry(zysk_okien_aluminiowych = 3000,
              zysk_okien_drewnianych = 5000,
              tempo_drewn_f1 = 0, #ile czasu musi poswiecic fabryka 1, aby mogly powstac drewniane okna
              tempo_drewn_f2 = 2, #ile czasu musi poswiecic fabryka 2, aby mogly powstac drewniane okna
              tempo_drewn_f3 = 2, #ile czasu musi poswiecic fabryka 3, aby mogly powstac drewniane okna
              tempo_alumi_f1 = 1, #ile czasu musi poswiecic fabryka 1, aby mogly powstac aluminiowe okna
              tempo_alumi_f2 = 0, #ile czasu musi poswiecic fabryka 2, aby mogly powstac aluminiowe okna
              tempo_alumi_f3 = 3, #ile czasu musi poswiecic fabryka 3, aby mogly powstac aluminiowe okna
              #aby powstaly okna, wszystkie fabryki je produkujace MUSZA pracowac nad nimi przez dana liczbe godzin!
              czas_f1 = 4,
              czas_f2 = 12,
              czas_f3 = 18):
    
    zyskOkna = LpProblem("Maksyamalizacja zysku z okien",LpMaximize)

    #zysk za x1 to 3000, a za x2 to 5000
    x1=LpVariable("oknaAluminiowe",0,None,LpInteger)
    x2=LpVariable("oknaDrewniane",0,None,LpInteger)
    
    #wspolczynnik do minimalizacji
    zyskOkna += zysk_okien_aluminiowych*x1 + zysk_okien_drewnianych*x2, "Zysk"

    #ograniczenia
    zyskOkna += x1*tempo_alumi_f1 + x2*tempo_drewn_f1 <= czas_f1, "CzasF1"
    zyskOkna += x1*tempo_alumi_f2 + x2*tempo_drewn_f2 <= czas_f2, "CzasF2"
    zyskOkna += x1*tempo_alumi_f3 + x2*tempo_drewn_f3 <= czas_f3, "CzasF3"

    #rozwiazanie
    zyskOkna.solve()
    print("Status:", LpStatus[zyskOkna.status])
    
    for v in zyskOkna.variables():
        print(v.name, "=", v.varValue)
    print("Zysk wyniesie {} zł  ".format(value(zyskOkna.objective)))
    
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in zyskOkna.constraints.items()]
    print("*"*10+"Shadow Prices"+"*"*10)   #slack - jak daleko jestesmy od ograniczenia
    print(pd.DataFrame(shadows))
    
interact(parametry, tempo_drewn_f1 = tempo_drewn_f1_slider,
             tempo_drewn_f2 = tempo_drewn_f2_slider,
             tempo_drewn_f3 = tempo_drewn_f3_slider,
             tempo_alumi_f1 = tempo_alumi_f1_slider,
             tempo_alumi_f2 = tempo_alumi_f2_slider,
             tempo_alumi_f3 = tempo_alumi_f3_slider,
             czas_f1 = czas_f1_slider,
             czas_f2 = czas_f2_slider,
             czas_f3 = czas_f3_slider)

interactive(children=(IntSlider(value=3000, description='zysk_okien_aluminiowych', max=9000, min=-3000), IntSl…

<function __main__.parametry(zysk_okien_aluminiowych=3000, zysk_okien_drewnianych=5000, tempo_drewn_f1=0, tempo_drewn_f2=2, tempo_drewn_f3=2, tempo_alumi_f1=1, tempo_alumi_f2=0, tempo_alumi_f3=3, czas_f1=4, czas_f2=12, czas_f3=18)>